In [ ]:
import numpy as np 
import pandas as pd 
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import Audio
import tensorflow as tf
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import random
import tqdm
from transformers import AutoModel, AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv('../input/contradictory-my-dear-watson/train.csv')

In [ ]:
data.head()

In [ ]:
data.isnull().any()

In [ ]:
data_dir = '../input/contradictory-my-dear-watson/'
train = pd.read_csv(data_dir + 'train.csv')
test = pd.read_csv(data_dir + 'test.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')
premise=data['premise']

In [ ]:
model = AutoModel.from_pretrained('joeddav/xlm-roberta-large-xnli')
tokenizer = AutoTokenizer.from_pretrained('joeddav/xlm-roberta-large-xnli')

In [ ]:
!pip install nlp
from nlp import load_dataset

In [ ]:
def load_mnli(use_validation=True):
    result = []
    dataset = load_dataset('multi_nli')
    print(dataset['train'])
    keys = ['train', 'validation_matched','validation_mismatched'] if use_validation else ['train']
    for k in keys:
        for record in dataset[k]:
            c1, c2, c3 = record['premise'], record['hypothesis'], record['label']
            if c1 and c2 and c3 in {0,1,2}:
                result.append((c1,c2,c3,'en'))
    result = pd.DataFrame(result, columns=['premise','hypothesis', 'label','lang_abv'])
    return result

In [ ]:
mnli = load_mnli()

In [ ]:
total_train = train[['id', 'premise', 'hypothesis','lang_abv', 'language', 'label']]
total_train

In [ ]:
mnli = mnli[['premise', 'hypothesis', 'lang_abv', 'label']]
mnli.insert(0, 'language', 'English')
mnli = mnli[['premise', 'hypothesis', 'lang_abv', 'language', 'label']]
mnli.insert(0, 'id', 'xxx')
mnli

In [ ]:
total_train = pd.concat([total_train, mnli], axis = 0)
total_train

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
class CLRPDataset(nn.Module):
    def __init__(self, df, tokenizer, max_len=256):
        self.excerpt = df['premise'].to_numpy()
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        
        
        
        
        
        
        return encode
    
    def __len__(self):
        return len(self.excerpt)
    
def get_embeddings(df, path, plot_losses=True, verbose=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used")
    model.to(device)
    model.eval()
    
 
    ds = CLRPDataset(df, tokenizer)
    dl = DataLoader(ds,
                    batch_size=config["batch_size"],
                    shuffle=False,
                    num_workers = 4,
                    pin_memory=True,
                    drop_last=False)
        
    embeddings = list()
    with torch.no_grad():
        for i, inputs in enumerate(dl):

            inputz = {key:val.reshape(val.shape[0], -1).to(device) for key, val in inputs.items()}
            outputs = model(**inputz)
            outputs = outputs[0][:, 0,:].detach().cpu().numpy()
            embeddings.extend(outputs)
    return np.array(embeddings)

config = {
    'batch_size': 128,
    'max_len': 256,
    'seed': 42,
} 
seed_everything(seed=config['seed'])

train_embeddings_premise =  get_embeddings(train,'../input/modelf1')
test_embeddings_premise = get_embeddings(test,'../input/modelf1')


In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
class CLRPDataset(nn.Module):
    def __init__(self, df, tokenizer, max_len=256):
        self.excerpt = df['hypothesis'].to_numpy()
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        
        
        
        
        
        
        return encode
    
    def __len__(self):
        return len(self.excerpt)
    
def get_embeddings(df, path, plot_losses=True, verbose=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used")
    model.to(device)
    model.eval()
    
 
    ds = CLRPDataset(df, tokenizer)
    dl = DataLoader(ds,
                    batch_size=config["batch_size"],
                    shuffle=False,
                    num_workers = 4,
                    pin_memory=True,
                    drop_last=False)
        
    embeddings = list()
    with torch.no_grad():
        for i, inputs in enumerate(dl):

            inputz = {key:val.reshape(val.shape[0], -1).to(device) for key, val in inputs.items()}
            outputs = model(**inputz)
            outputs = outputs[0][:, 0,:].detach().cpu().numpy()
            embeddings.extend(outputs)
    return np.array(embeddings)

config = {
    'batch_size': 128,
    'max_len': 256,
    'seed': 42,
} 
seed_everything(seed=config['seed'])

train_embeddings_hypo =  get_embeddings(train,'../input/modelf1')
test_embeddings_hypo = get_embeddings(test,'../input/modelf1')


In [ ]:
from keras.utils.np_utils import to_categorical 
y=train['label']
y = to_categorical(y)

In [ ]:
y

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
class CLRPDataset(nn.Module):
    def __init__(self, df, tokenizer, max_len=256):
        self.excerpt = df['premise'].to_numpy()
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        
        
        
        
        
        
        return encode
    
    def __len__(self):
        return len(self.excerpt)
    
def get_embeddings(df, path, plot_losses=True, verbose=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used")
    model.to(device)
    model.eval()
    
 
    ds = CLRPDataset(df, tokenizer)
    dl = DataLoader(ds,
                    batch_size=config["batch_size"],
                    shuffle=False,
                    num_workers = 4,
                    pin_memory=True,
                    drop_last=False)
        
    embeddings = list()
    with torch.no_grad():
        for i, inputs in enumerate(dl):

            inputz = {key:val.reshape(val.shape[0], -1).to(device) for key, val in inputs.items()}
            outputs = model(**inputz)
            outputs = outputs[0][:, 0,:].detach().cpu().numpy()
            embeddings.extend(outputs)
    return np.array(embeddings)

config = {
    'batch_size': 128,
    'max_len': 256,
    'seed': 42,
} 
seed_everything(seed=config['seed'])

total_train_embeddings_premise =  get_embeddings(total_train,'../input/modelf1')

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
class CLRPDataset(nn.Module):
    def __init__(self, df, tokenizer, max_len=256):
        self.excerpt = df['hypothesis'].to_numpy()
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        
        
        
        
        
        
        return encode
    
    def __len__(self):
        return len(self.excerpt)
    
def get_embeddings(df, path, plot_losses=True, verbose=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used")
    model.to(device)
    model.eval()
    
 
    ds = CLRPDataset(df, tokenizer)
    dl = DataLoader(ds,
                    batch_size=config["batch_size"],
                    shuffle=False,
                    num_workers = 4,
                    pin_memory=True,
                    drop_last=False)
        
    embeddings = list()
    with torch.no_grad():
        for i, inputs in enumerate(dl):

            inputz = {key:val.reshape(val.shape[0], -1).to(device) for key, val in inputs.items()}
            outputs = model(**inputz)
            outputs = outputs[0][:, 0,:].detach().cpu().numpy()
            embeddings.extend(outputs)
    return np.array(embeddings)

config = {
    'batch_size': 128,
    'max_len': 256,
    'seed': 42,
} 
seed_everything(seed=config['seed'])

total_train_embeddings_hypo =  get_embeddings(total_train,'../input/modelf1')

In [ ]:
total_y= total_train['label']
total_y = to_categorical(total_y)

In [ ]:
train_embeddings_premise.shape

In [ ]:
from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM,Dropout,concatenate
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Activation, Embedding, LSTM,Dropout,Bidirectional,GRU
from keras.utils import plot_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Flatten ,Embedding,Input,Conv1D,GlobalAveragePooling1D,GlobalMaxPooling1D,Dropout,MaxPooling1D,Bidirectional,GRU,Concatenate
from keras.models import Sequential,Model

In [ ]:
def crt_model():
    i1=Input(shape=(1024,1))
    l1=Conv1D(64,5,padding='valid', kernel_initializer='normal',activation='relu')(i1)
    l2 =MaxPooling1D(2) (l1)
    l3=Conv1D(128,5,padding='valid', kernel_initializer='normal',activation='relu')(l2)
    l3 =MaxPooling1D(2) (l3)
    l3=Conv1D(256,5,padding='valid', kernel_initializer='normal',activation='relu')(l3)
    l4=GlobalMaxPooling1D()(l3)
    
    
    i12=Input(shape=(1024,1))
    l12=Conv1D(64,5,padding='valid', kernel_initializer='normal',activation='relu')(i12)
    l22 =MaxPooling1D(2) (l12)
    l32=Conv1D(128,5,padding='valid', kernel_initializer='normal',activation='relu')(l22)
    l32 =MaxPooling1D(2) (l32)
    l32=Conv1D(256,5,padding='valid', kernel_initializer='normal',activation='relu')(l32)
    l42=GlobalMaxPooling1D()(l32)
    
    l5=concatenate([l4,l42])
    
    l5=Dense(120, kernel_initializer='normal',activation='relu')(l5)
    l5=Dense(240, kernel_initializer='normal',activation='relu')(l5)
    l7=Dense(3, kernel_initializer='normal',activation='softmax')(l5)
    model=Model(inputs=[i1,i12], outputs=l7)
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc'])
    return model

In [ ]:
model=crt_model()
model.summary()

In [ ]:
plot_model(model, to_file='multiple_inputs.png')

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
nfolds = 5
kf = KFold(n_splits=nfolds, shuffle=True, random_state=config['seed'])
best_iterations = []
oof_rmses = []
preds = np.zeros(test.shape[0])
for k, (train_idx, valid_idx) in enumerate(kf.split(train)): 
    print(f'Fold {k}')
    train_x_prm,train_x_hypo,train_y,test_x_prm,test_x_hypo,test_y=total_train_embeddings_premise[train_idx],total_train_embeddings_hypo[train_idx], total_y[train_idx],total_train_embeddings_premise[valid_idx],total_train_embeddings_hypo[valid_idx], total_y[valid_idx]
    model.fit([train_x_prm.reshape(train_x_prm.shape+(1,)),train_x_hypo.reshape(train_x_hypo.shape+(1,))],
              train_y,
              epochs=40,
              validation_data=([test_x_prm, test_x_hypo],test_y),
              batch_size=32)
    y_pred=model.predict([test_embeddings_premise,test_embeddings_hypo])
    y_pred=y_pred.reshape(-1)
    preds+=y_pred
    
preds = np.array(preds)/nfolds
    

In [ ]:
submission = pd.DataFrame({'id':dt.id,'prediction':preds})
submission.to_csv('submission.csv',index=False)